In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install flask-ngrok

Mounted at /content/drive


## Database Connection

In [ ]:
# import mysql.connector
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="yourusername",
#   password="yourpassword"
# )

# print(mydb)

In [ ]:
import os
#os.chdir('/content/drive/MyDrive/Dan/Dan')
os.chdir('/content/drive/MyDrive/Work/Dan/Deployment')
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
#library imports
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in re.split('\W+', text) if word not in stopword]
    return text

ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return ' '.join(text)

def spell_correction(text):           # spelling correction
    txt=TextBlob(text)
    return txt.correct()

def clean_data(x):
    x=x.lower()
    x=x.encode('ascii','ignore').decode() # remove texts other than english
    x=re.sub('https*\S+','',x) # remove urls
    #x=spell_correction(x)
    x=remove_punct(x) # remove punctuations
    x=remove_stopwords(x) # remove stopwords
    #x=stemming(x) # stemming
    #x=lemmatizer(x) # lemmatization
    return ' '.join(x)
    
tok = spacy.load('en')

def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

def encode_sentence(text, vocab2index, N=10):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length
    
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

def prep_data(text):
  text=clean_data(text)
  text=np.array(encode_sentence(text,vocab2index))
  return text

class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True) # activation is taken as tanh
        self.linear = nn.Linear(hidden_dim, df_train.label.nunique())
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])
import pickle
with open('model_files/label_encoder.pkl', 'rb') as handle:
    le= pickle.load(handle)
with open('model_files/labels.pkl', 'rb') as handle:
    labels=pickle.load(handle)
model = torch.load('model_files/model.pth')

with open('model_files/vocab.pkl','rb') as vocab:
    vocab2index=pickle.load(vocab)

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from string import Template
from flask import Flask, render_template, request

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
	return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
  if request.method=='POST':
    message = request.form['message']
    x=torch.from_numpy(prep_data(message)[0].astype(np.int32)).reshape(1,-1).long().to(device)
    pred=le.inverse_transform(np.argmax(model(x,None).cpu().detach().numpy(),axis=1))[0]   
    print(pred) 
  return render_template('result.html',prediction = pred)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e49c486235c3.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Feb/2021 04:26:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 04:26:06] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2021 04:26:06] "GET /favicon.ico HTTP/1.1" 404 -


computer.machine_crash


127.0.0.1 - - [25/Feb/2021 04:26:12] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 04:26:12] "GET /static/css/styles.css HTTP/1.1" 404 -
